In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import *
from PIL import Image
from torchvision import transforms
import torchvision.transforms.functional as tf
import os
from torch.optim import RMSprop
from torchvision.utils import make_grid
from pylab import plt
import Mod.res_block as resblock

In [2]:
#参数配置
#path=os.path.abspath('../../dataset/Animals_with_Attributes2/')
tar_path=os.path.abspath('../../dataset/GZSL_for_AWA/Animals_with_Attributes2/')
bat_size=40
img_size=227
gpu=True
tr_epoch=1
worker=2


transform1=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3,[0.5]*3)
])

In [3]:
#数据集载入
class SeenData(Dataset):
    def __init__(self,root,transforms=None,train_data=True,label=None):
        #路径
        #self.path=root
        
        if train_data==True:
            #root_label=os.path.abspath('../dataset/Animals_with_Attributes2')
            label_path=[x.path for x in os.scandir(root) if 'trainclass' in x.path]
            word=[]
            for i,file_label in enumerate(label_path):
                f=open(file_label,'r')
                while True:
                    temp_w=f.readline().strip()
                    if not temp_w:
                        break
                    word.append(temp_w)
                f.close()
            
            path_dir=[]
            self.label_list=word
            for key_word in self.label_list:
                [path_dir.append(x.path) for x in os.scandir(root+'/trainingset/') if x.name.endswith(key_word)]
                
            pic_path=[]
            label_t=[]
            for l,train_root in enumerate(path_dir):#对训练集文件夹遍历
                #[pic_path.append(x.path) for x in os.scandir(train_root) if x.name.endswith(".jpg")]#每个文件夹里的图片地址
                for x in os.scandir(train_root):
                    if x.name.endswith(".jpg"):
                        pic_path.append(x.path)
                        label_t.append(l)
                        
            #pic_path=path_dir#图片文件夹目录
            label_np=np.array(label_t)
            label_tensor=torch.from_numpy(label_np).type(torch.LongTensor)
            self.images_files=pic_path #图片目录
            self.transforms=transforms
            self.label=label_tensor
            
        else:
            self.path#测试机读取
            
    def __getitem__(self,index):
        try:
            pic_data=self.transforms(Image.open(self.images_files[index]))
            
        except RuntimeError:
            print(self.images_files[index])
            return self.transforms(Image.open(self.images_files[index-1])),self.label[index]
            
        else:
            return pic_data,self.label[index]
        
    def __len__(self):
        return len(self.images_files)
        

In [3]:
#数据集载入
class SeenData(Dataset):
    def __init__(self,root,transforms=None,train_data=True,label=None):
        #路径
        #self.path=root
        
        if train_data==True:
            dir_class='trainingset'
            #root_label=os.path.abspath('../dataset/Animals_with_Attributes2')
        else:
            dir_class='testingset'
                
        label_path=[x.path for x in os.scandir(root) if 'trainclass' in x.path]
        word=[]
        for i,file_label in enumerate(label_path):
            f=open(file_label,'r')
            while True:
                temp_w=f.readline().strip()
                if not temp_w:
                     break
                word.append(temp_w)
            f.close()
            
        path_dir=[]
        self.label_list=word
        for key_word in self.label_list:
            print(key_word)
            [path_dir.append(x.path) for x in os.scandir(root+'/'+dir_class+'/') if x.name.endswith(key_word)]
                
        pic_path=[]
        label_t=[]
        for l,train_root in enumerate(path_dir):#对训练集文件夹遍历
            #[pic_path.append(x.path) for x in os.scandir(train_root) if x.name.endswith(".jpg")]#每个文件夹里的图片地址
            for x in os.scandir(train_root):
                if x.name.endswith(".jpg"):
                    pic_path.append(x.path)
                    label_t.append(l)
                        
            #pic_path=path_dir#图片文件夹目录
        label_np=np.array(label_t)
        label_tensor=torch.from_numpy(label_np).type(torch.LongTensor)
        self.images_files=pic_path #图片目录
        self.transforms=transforms
        self.label=label_tensor
        
    def __getitem__(self,index):
        try:
            pic_data=self.transforms(Image.open(self.images_files[index]))
            
        except RuntimeError:
            print(self.images_files[index])
            return self.transforms(Image.open(self.images_files[index-1])),self.label[index]
            
        else:
            return pic_data,self.label[index]
        
    def __len__(self):
        return len(self.images_files)
        

In [4]:
#网络
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


In [5]:
alex=AlexNet(num_classes=40)
loss_func=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(params=alex.parameters(),lr=0.001)
indata=SeenData(tar_path,transform1)
data_training=DataLoader(indata,bat_size,shuffle = True,num_workers=worker)
data_testing=DataLoader(indata,bat_size,shuffle = False,num_workers=worker)
#resb=resblock.ResBlock()
if gpu:
    
    alex.cuda()
    

antelope
grizzly+bear
killer+whale
beaver
dalmatian
horse
german+shepherd
blue+whale
siamese+cat
skunk
mole
tiger
moose
spider+monkey
elephant
gorilla
ox
fox
sheep
hamster
squirrel
rhinoceros
rabbit
bat
giraffe
wolf
chihuahua
weasel
otter
buffalo
zebra
deer
bobcat
lion
mouse
polar+bear
collie
walrus
cow
dolphin


In [6]:
#training
Loss=[]
Test_Loss=[]
for epoch in range(tr_epoch):
    training_loss=0.0
    testing_error=0.0
    for iter_batch,batch in enumerate(data_training):
        
        data=batch[0]
        tar_label=batch[1]
        #print(tar_label)
        if gpu:
            data=data.cuda()
            tar_label=tar_label.cuda()
        output=alex(data)
        #print(output.cpu())
        loss=loss_func(output,tar_label)
        #print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        training_loss+= loss.item()# * batch.size(0)
        
    print(training_loss)
    Loss.append(training_loss)
        
        
    if epoch%400==0:
        torch.save(alex.state_dict(),'./models/'+str(epoch+1)+'seenCNN.pkl')
        
    #测试误差
    error_sam=0.0
    total_sam=0.0
    for iter_num,t_batch in enumerate(data_testing):
        data_t=t_batch[0]
        label_t=t_batch[1]
        if gpu:
            data_t=data_t.cuda()
        output_t=alex(data_t)
        print('result of prediction')
        print(output_t.cpu().argmax(dim=1))
        print('ground truth')
        print(label_t)
        error_mat=output_t.cpu().argmax(dim=1)-label_t
        error_sam=error_sam+error_mat[error_mat!=0].size()[0]
        total_sam=total_sam+data_t.size()[0]
        
    error_rate=error_sam/total_sam
    print(error_rate)
    Test_Loss.append(error_rate)
    
Loss0 = np.array(Loss)
print(Loss0)
#torch.save(Loss0,'./models/train_loss.pth')  
#np.save('./models/train_loss.npy',Loss0)

RuntimeError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 7.92 GiB total capacity; 242.08 MiB already allocated; 16.06 MiB free; 254.00 MiB reserved in total by PyTorch)